In [1]:
with open('sample_name.txt','r') as f:
    name = f.read().split("\n")
    name_dict = {}
    for i in name:
        key, value = i.split(":")
        name_dict[key] = value
    number_of_samples = len(name_dict)

# 資料預處理

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,      
                                   zoom_range = 0.2,       
                                   horizontal_flip = True) 
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

Using TensorFlow backend.


In [3]:
training_set = train_datagen.flow_from_directory('trainset/', target_size = (64, 64), batch_size = 10,
                                                 class_mode = 'categorical')
validation_set = validation_datagen.flow_from_directory('validationset/', target_size = (64, 64), batch_size = 10, 
                                                        class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('testset/', target_size = (64, 64), batch_size = 10, 
                                            class_mode = 'categorical')

Found 204 images belonging to 3 classes.
Found 52 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


# 建立模型

In [4]:
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2))) 
classifier.add(Conv2D(32, (3, 3), activation = 'relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2))) 
classifier.add(Flatten()) 
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = number_of_samples, activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

# 訓練模型

In [5]:
# try:
#     classifier.load_weights("SaveModel/face.h5")
#     print("載入模型成功!繼續訓練模型")
# except :    
#     print("載入模型失敗!開始訓練一個新模型")

In [6]:
history = classifier.fit_generator(training_set,
                                   nb_epoch=10,
                                   nb_val_samples=30,
                                   steps_per_epoch = 30,
                                   verbose = 1,
                                   validation_data = validation_set)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=30, verbose=1, validation_data=<keras.pre..., epochs=10, validation_steps=30)`
  


Epoch 1/10
30/30 [==============================] - 2s 81ms/step - loss: 0.5943 - acc: 0.7738 - val_loss: 0.0250 - val_acc: 1.0000
Epoch 2/10
30/30 [==============================] - 2s 57ms/step - loss: 0.0925 - acc: 0.9603 - val_loss: 0.1048 - val_acc: 0.9808
Epoch 3/10
30/30 [==============================] - 2s 64ms/step - loss: 0.0234 - acc: 0.9933 - val_loss: 0.0106 - val_acc: 1.0000
Epoch 4/10
30/30 [==============================] - 2s 58ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0104 - val_acc: 1.0000
Epoch 5/10
30/30 [==============================] - 2s 65ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0122 - val_acc: 1.0000
Epoch 6/10
30/30 [==============================] - 2s 61ms/step - loss: 7.6916e-04 - acc: 1.0000 - val_loss: 0.0218 - val_acc: 0.9808
Epoch 7/10
30/30 [==============================] - 2s 61ms/step - loss: 7.0678e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 8/10
30/30 [==============================] - 2s 65ms/step - loss: 1.

# 模型預測

In [7]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(r'sample0_face\sample0_0.png', target_size= (64,64))
test_image = image.img_to_array(test_image)
test_image.shape

(64, 64, 3)

In [8]:
test_image = np.expand_dims(test_image, axis = 0)
type(test_image)
test_image.shape

(1, 64, 64, 3)

In [9]:
classifier.predict_classes(test_image)

array([0], dtype=int64)

In [10]:
name_dict['sample'+str(classifier.predict_classes(test_image)[0])] 

'luokuofeng'

# 儲存模型

In [12]:
import os

if not os.path.exists('SaveModel'):
    os.mkdir('SaveModel')
classifier.save_weights("SaveModel/face.h5")
print("Saved model to disk")

Saved model to disk
